In [ ]:
import numpy as np
import torch
import matplotlib.pyplot as plt
import torch_geometric
from torch_geometric.loader import DataLoader
import pandas as pd
import random
import os
from torch_geometric.data import Data
import networkx as nx
from torch_geometric.utils import to_networkx
import torch.nn.functional as F
from torch_geometric.nn import GCNConv, global_mean_pool, NNConv, SAGEConv, BatchNorm
from tqdm import tqdm
import torch.optim as optim
import torch.optim.lr_scheduler as lr_scheduler

In [ ]:
os.getcwd()
use_cuda_if_available = False
BATCH_SIZE = 30
device = torch.device('cuda' if torch.cuda.is_available() and use_cuda_if_available else 'cpu')

In [ ]:
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed) 
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False 
    
seed = 1234
set_seed(seed)

# Importing the Main tabular data

In [ ]:
signal = pd.read_csv('/teamspace/studios/this_studio/GNN/data/signal_gnn.txt', sep = '\t')
background = pd.read_csv('/teamspace/studios/this_studio/GNN/data/background_gnn.txt', sep = '\t')

In [ ]:
signal.head()

In [ ]:
background.columns

# Plan for defining the Graph

Individual Event -> Graph -> Fully Connected Graph, Undirectional

Objects(Isophoton, jet1, jet2) -> Nodes

Node Lable -> (isophoton, jet1, jet2) -> (0,1,2)

Node Features -> (Eta, Phi, Px, Py, Pz, E)

Edge Index -> ((0, 1, 2),(1, 2, 0))

Edge Features -> $\Delta R$ between the ((0,1), (1,2), (2,0))

Graph Features -> (jet_multiplicity, inv_mass)

Graph Lable -> 0:Background, 1:Signal

# Aggregating Node Features

In [ ]:
def get_node_features(x:pd.DataFrame) -> list:
    """
    Extracts node features for isophoton, jet1, and jet2 from a DataFrame
    and returns a list of node features for all events.

    Parameters:
    -----------
    x : pd.DataFrame
        Input DataFrame with columns containing features for each object in the event.

    Returns:
    --------
    list_of_node_features : list of lists
        Each element in the list is a 2D list of shape (3, num_features) 
        containing node features for isophoton, jet1, and jet2.
    """
    # List to store node features for all events (graphs)
    list_of_node_features = []

    # Columns for each object's features
    isophoton_cols = ['isophoton_Eta', 'isophoton_Phi', 
                      'isophoton_Px', 'isophoton_Py', 'isophoton_Pz', 'isophoton_E']
    jet1_cols = ['jet1_Eta', 'jet1_Phi', 
                 'jet1_Px', 'jet1_Py', 'jet1_Pz', 'jet1_E']
    jet2_cols = ['jet2_Eta', 'jet2_Phi', 
                 'jet2_Px', 'jet2_Py', 'jet2_Pz', 'jet2_E']

    for _, event in x.iterrows():
        isophoton_features = event[isophoton_cols].values.tolist()  # List of features for isophoton
        jet1_features = event[jet1_cols].values.tolist()            # List of features for jet1
        jet2_features = event[jet2_cols].values.tolist()            # List of features for jet2

        event_node_features = [isophoton_features, jet1_features, jet2_features]

        list_of_node_features.append(event_node_features)

    return list_of_node_features

In [ ]:
signal_node_features_list = get_node_features(signal)
background_node_features_list = get_node_features(background)

In [ ]:
# Shape of node_features_list
print(len(signal_node_features_list))           # Number of events (graphs)
print(len(signal_node_features_list[0]))        # Number of nodes (3: isophoton, jet1, jet2)
print(len(signal_node_features_list[0][0]))     # Number of features per node

In [ ]:
# Shape of node_features_list
print(len(background_node_features_list))           # Number of events (graphs)
print(len(background_node_features_list[0]))        # Number of nodes (3: isophoton, jet1, jet2)
print(len(background_node_features_list[0][0]))     # Number of features per node

In [ ]:
signal_node_features_list[0] #Each Row represents node feature

# Aggregating Edge Index and Edge Features

In [ ]:
edge_index = [[0,1,2],[1,2,0]]  #Directed edge (0 -> 1, 1 -> 2, 2 -> 0)

def get_edge_features(x:pd.DataFrame) -> list:
    """
    Extracts edge features(deltaR) between isophoton, jet1, and jet2 from a DataFrame
    and returns a list of edge features for all events.

    Parameters:
    -----------
    x : pd.DataFrame
        Input DataFrame with columns containing features for each object in the event.

    Returns:
    --------
    list_of_edge_features : list of lists
        Each element in the list is a 1D list of size 3 
        containing edge features for isophoton, jet1, and jet2.
    """

    list_of_edge_features = []

    for _, event in x.iterrows():
        edge_01 = event['deltaR_iso_jet1']
        edge_12 = event['deltaR_jet12']
        edge_20 = event['deltaR_iso_jet2']
        
        list_of_edge_features.append([edge_01, edge_12, edge_20])
    return list_of_edge_features

In [ ]:
signal_edge_features_list = get_edge_features(signal)
background_edge_features_list = get_edge_features(background)

In [ ]:
print(len(signal_edge_features_list))
print(len(signal_edge_features_list[0]))

In [ ]:
signal_edge_features_list[0]

In [ ]:
print(len(background_edge_features_list))
print(len(background_edge_features_list[0]))

# Aggregating Data for Graphs

In [ ]:
def get_graph_features(node_features_list:list, edge_features_list:list, edge_index_list:list) -> dict:
    '''
    Extract node features,  edge features and edge index for each event and put then into a dictionary
    representating each event and return a list containing the dictionaries.

    Parameters:
    -----------
    node_features_list : list
        Each element in the list is a 2D list of shape (3, num_features) 
        containing node features for isophoton, jet1, and jet2.

    edge_features_list : list
        Each element in the list is a 1D list of size 3 
        containing edge features for isophoton, jet1, and jet2.

    edge_index_list : list
        2D list of shape (2, num_edges), connectivity information.

    Returns:
    --------
    event_graph : list of dictionaries
        Each element contains:
            - 'nodes': 2D list of shape (3, num_features), node features for isophoton, jet1, and jet2.
            - 'edges': 2D list of shape (num_edges, edge_features), edge features (Delta R) for the fully connected graph.
            - 'edge_index': 2D list of shape (2, num_edges), connectivity information.
    '''
    event_graphs = []
    if len(node_features_list) == len(edge_features_list):
        for i in range(len(node_features_list)):
            graph_dict = {
                'nodes' : node_features_list[i],
                'edges' : edge_features_list[i],
                'edge_index' : edge_index_list
            }
            event_graphs.append(graph_dict)
        return event_graphs

    else:
        print('Number of Events in lists dont match')

In [ ]:
signal_graphs = get_graph_features(signal_node_features_list, signal_edge_features_list, edge_index)
background_graphs = get_graph_features(background_node_features_list, background_edge_features_list, edge_index)

In [ ]:
print(len(signal_graphs))
print(len(signal_graphs[0]))

In [ ]:
signal_graphs[0]

# Converting Each Event into a PyG Graph Object

In [ ]:
import torch
from torch_geometric.data import Data

def convert_to_pyg_graph(graph, jet_multiplicity, inv_mass, deltaR_jet12, label):
    """
    Convert the graph dictionary into a PyTorch Geometric Data object
    and add graph-level features like jet_multiplicity, inv_mass, and the event label.

    Parameters:
    -----------
    graph : dict
        Dictionary containing 'nodes', 'edges', and 'edge_index' for a single event.
    jet_multiplicity : float
        The number of jets in the event.
    inv_mass : float
        The invariant mass of the event.
    label : int
        The label of the event (1 for signal, 0 for background).

    Returns:
    --------
    data : Data
        PyTorch Geometric Data object containing the graph and additional features.
    """
    # Extract node features, edge features, and edge index from the graph dictionary
    node_features = torch.tensor(graph['nodes'], dtype=torch.float)  # Shape: (3, num_features)
    edge_index = torch.tensor(graph['edge_index'], dtype=torch.long)  # Shape: (2, num_edges)
    edge_features = torch.tensor(graph['edges'], dtype=torch.float).view(-1,1)  # Shape: (num_edges, 1)

    # Convert graph-level features (jet_multiplicity, inv_mass) and label to tensor
    graph_level_features = torch.tensor([jet_multiplicity, inv_mass, edge_features.std().item(), deltaR_jet12], dtype=torch.float).view(1, -1)
    label_tensor = torch.tensor([label], dtype=torch.long)  # Event label (0 or 1)

    # Create the PyTorch Geometric Data object
    data = Data(
        x=node_features,               # Node features (3, num_features)
        edge_index=edge_index,         # Edge index (2, num_edges)
        edge_attr=edge_features,       # Edge features (num_edges, 1)
        y=label_tensor                 # Event label (0 or 1)
    )

    # Add custom graph-level features
    data.jet_multiplicity = graph_level_features[0, 0]
    data.inv_mass = graph_level_features[0, 1]
    data.proximity_spread = graph_level_features[0, 2]
    data.deltaR_jet12 = graph_level_features[0, 3]

    return data

def convert_all_to_pyg_graphs(graphs, dataframe):
    """
    Converts a list of event graphs to PyTorch Geometric-compatible Data objects.
    Also adds the graph-level features 'jet_multiplicity', 'inv_mass' and event labels.

    Parameters:
    -----------
    graphs : list of dicts
        List containing dictionaries with node, edge information for each event.
    dataframe : pd.DataFrame
        DataFrame containing the event-level features like 'jet_multiplicity', 'inv_mass', and 'label'.

    Returns:
    --------
    pyg_graphs : list of Data
        List of PyTorch Geometric Data objects with added graph-level features and labels.
    """
    pyg_graphs = []
    for i, graph in enumerate(graphs):
        # Extract jet_multiplicity, inv_mass and event label for each event
        jet_multiplicity = dataframe.loc[i, 'jet_multiplicity']
        inv_mass = dataframe.loc[i, 'inv_mass']
        deltaR_jet12 = dataframe.loc[i, 'deltaR_jet12']
        label = dataframe.loc[i, 'label']  # 0 for background, 1 for signal

        # Convert to PyTorch Geometric Data object
        data = convert_to_pyg_graph(graph, jet_multiplicity, inv_mass, deltaR_jet12, label)
        pyg_graphs.append(data)

    return pyg_graphs


signal_pyg_graphs = convert_all_to_pyg_graphs(signal_graphs, signal)
background_pyg_graphs = convert_all_to_pyg_graphs(background_graphs, background)

In [ ]:
dataset = signal_pyg_graphs + background_pyg_graphs
dataset = random.sample(dataset, len(dataset))

In [ ]:
dataset[2000]

# Train-Test Splits

In [ ]:
num_graphs = len(dataset)
train_ratio, test_ratio = 0.7, 0.3
train_size = int(train_ratio * num_graphs)
test_size = num_graphs - train_size

# Random split
train_dataset, test_dataset = torch.utils.data.random_split(
    dataset, [train_size, test_size], generator=torch.Generator().manual_seed(seed)
)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)


In [ ]:
len(train_loader)

In [ ]:
len(dataset)

# Graph Visualization

In [ ]:
def visualize_graph(data, title):
    """
    Visualize a graph using networkx and matplotlib.
    
    Parameters:
    -----------
    data : Data
        A PyTorch Geometric Data object containing the graph to be visualized.
    """
    G = to_networkx(data, to_undirected=True, edge_attrs=["edge_attr"])

    
    plt.figure(figsize=(5, 3))
    
    pos = nx.spring_layout(G)  # Layout for positioning nodes
    nx.draw(G, pos, with_labels=True, node_size=300, node_color='skyblue', font_size=10, font_weight='bold', edge_color='gray')

    edge_labels = nx.get_edge_attributes(G, 'edge_attr')
    nx.draw_networkx_edge_labels(G, pos, edge_labels=edge_labels)

    plt.title(title)
    plt.show()


visualize_graph(signal_pyg_graphs[0], 'Signal')
visualize_graph(background_pyg_graphs[0], 'Background')

## Model 4 Best

In [ ]:
class GCNClassifier(torch.nn.Module):
    def __init__(self, num_node_features, num_edge_features, hidden_dim, num_graph_features, num_hidden_graph_features):
        """
        GCN architecture for graph classification.

        Parameters:
        -----------
        num_node_features : int
            Number of features for each node.
        hidden_dim : int
            Dimension of the hidden layer embeddings.
        num_graph_features : int
            Number of additional graph-level features.
        """
        super(GCNClassifier, self).__init__()

        self.classify = torch.nn.Sequential(
            torch.nn.Linear(hidden_dim + num_hidden_graph_features, 64),
            torch.nn.ReLU(),
            torch.nn.Dropout(p=0.5),
            torch.nn.Linear(64, 1)
        )

        self.graph_features_nn = torch.nn.Sequential(
            torch.nn.Linear(num_graph_features, 16),
            torch.nn.ReLU(),
            torch.nn.Dropout(p=0.7),
            torch.nn.Linear(16, num_hidden_graph_features)
        )


        ############### Model 4 - Best
        self.lin0 = torch.nn.Linear(num_node_features, hidden_dim)
        self.edge_nn = torch.nn.Sequential(
            torch.nn.Linear(num_edge_features, 16),
             torch.nn.ReLU(),
             torch.nn.Linear(16, hidden_dim * hidden_dim)
        )
        
        self.conv2 = NNConv(hidden_dim, hidden_dim, nn = self.edge_nn, aggr='max')
        self.global_mean_pool_hook = global_mean_pool
        

    def forward(self, data):
        """
        Forward pass of the GCN model.

        Parameters:
        -----------
        data : Data
            PyTorch Geometric Data object containing graph and its features.

        Returns:
        --------
        out : torch.Tensor
            Predicted probabilities for the graph labels.
        """
        x, edge_index, edge_attr = data.x, data.edge_index, data.edge_attr
        
        # GCN layers
        x = F.relu(self.lin0(x))
        x = F.relu(self.conv2(x, edge_index, edge_attr))
        
        x = self.global_mean_pool_hook(x, data.batch)  # Shape: (batch_size, hidden_dim)
        self.global_embedding = x

        graph_features = torch.cat([data.jet_multiplicity.view(-1, 1), data.inv_mass.view(-1, 1)], dim=1)
        x = torch.cat([x, self.graph_features_nn(graph_features)], dim=1)
        
        # Fully connected layers
        x = self.classify(x)
        x = torch.sigmoid(x) 

        return x


# Training and Evaluation Routines

In [ ]:
def train(model, loader):
    model.train()
    total_loss = 0
    for data in loader:
        data = data.to(device)
        optimizer.zero_grad()
        out = model(data).squeeze()
        loss = criterion(out, data.y.float())
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    return total_loss / len(loader)

# Evaluation loop
def test(model, loader):
    model.eval()
    correct = 0
    for data in loader:
        data = data.to(device)
        out = model(data).squeeze()
        pred = (out > 0.5).float()
        correct += (pred == data.y).sum().item()
    return correct / len(loader.dataset)

## Network Training

In [ ]:
model = GCNClassifier(num_node_features=6, num_edge_features = 1, hidden_dim=49, num_graph_features=2, num_hidden_graph_features = 1)
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-4, amsgrad = False)
criterion = torch.nn.BCELoss()

EPOCHS = 250
training_loss = []
test_accuracy = []

with tqdm(total=EPOCHS, desc="Training Progress", leave=True) as pbar:
    for epoch in range(EPOCHS):
        loss = train(model, train_loader)
        acc = test(model, test_loader)
        training_loss.append(loss)
        test_accuracy.append(acc)

        pbar.set_postfix({'Loss': f'{loss:.4f}', 'Accuracy': f'{acc:.4f}'})
        pbar.update(1)

In [ ]:
plt.plot([epoch for epoch in range(EPOCHS)], training_loss, label = 'Training Loss')
plt.plot([epoch for epoch in range(EPOCHS)], test_accuracy, color = 'g', label = 'Test Accuracy')
plt.legend()
plt.title('Evaluation Metrices')
plt.xlabel('Epochs')
plt.grid(which = 'both')
#plt.savefig('94auc.png', dpi=200)
plt.show()

y_true = []
y_pred_prob = []
for _, graph in enumerate(dataset):
    y_true.append(graph.y.numpy()[0])
    out = model(graph).squeeze().detach().numpy()
    y_pred_prob.append(float(out))
y_true, y_pred_prob = np.array(y_true), np.array(y_pred_prob)
y_pred_labels = (y_pred_prob > 0.5)

In [ ]:
from helper_function_module import eval_model
eval_model(y_true, y_pred_prob, y_pred_labels)

In [ ]:
# np.save('NNConv_main_model', np.vstack((y_true, y_pred_prob)))
# torch.save(model, 'NNConv_main_model.pth')

# Extracting Graph Embedding from the Model

In [ ]:
def get_graph_embeddings():
    graph_embeddings = []
    def hook_fn(module, input, output):
        graph_embeddings.append(module.global_embedding.detach().cpu())
        
    
    model = GCNClassifier(num_node_features=6, num_edge_features=1, hidden_dim=49, 
                          num_graph_features=2, num_hidden_graph_features=1)
    
    handle = model.register_forward_hook(hook_fn)
    
    dataset_loader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=False)
    model.eval()
    y_true_label = []
    y_pred_proba = []
    with torch.no_grad():
        for data in dataset_loader:
            y_true_label.extend(data.y.numpy())
            out = model(data)
            y_pred_proba.extend(out.numpy())
    
    handle.remove()
    graph_embedding_temp = []
    for tensor in graph_embeddings:
        graph_embedding_temp.extend(tensor.numpy())

    graph_embedding_temp = np.array(graph_embedding_temp).reshape((2415, 49))
    embedding_df = pd.DataFrame(graph_embedding_temp)
    embedding_df['true_label'] = y_true_label
    embedding_df['pred_prob'] = y_pred_proba
    embedding_df['pred_prob'] = embedding_df['pred_prob'].astype(float)

    return embedding_df

In [ ]:
# embedding_df = get_graph_embeddings()